<a href="https://colab.research.google.com/github/SSobol/GNNCommunityDetection/blob/main/Embedding_NN_NetClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycombo
#!pip install fastnode2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Network community detection based on unsupervised node embedding clustering using Vanilla Neural Networks**

* Learn node embedding based on random walk probability distributions with home teleport

* Baseline community detection through embedding clustering

* VNN node clustering based on node embedding aiming to optimize modularity

 ** Unsupervised VNN training from scratch fails

 ** Supervised VNN pre-tune to match clustering results helps

 ** Further unsupervised finetune optimizing modularity can reach close to best known partition scores (not reached by clustering), but performance is rather unstable

In [2]:
import pycombo
#import fastnode2vec

In [3]:
import networkx as nx
import numpy as np
import time
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(sci_mode=False)

from tensorflow.keras.utils import to_categorical
from sklearn.mixture import GaussianMixture

In [4]:
#load other presaved networks
def loadNetworkMat(filename, path = 'SampleNetworks/ProcessedMat/'):
    A = scipy.io.loadmat(path + filename)
    if check_symmetric(A['net']):
        G = nx.from_numpy_matrix(A['net'])
    else:
        G = nx.from_numpy_matrix(A['net'], create_using=nx.DiGraph)
    return G

In [5]:
def residential_page_rank_embedding(A, alpha=0.85): #get node embedding vector as probabilities of visiting other nodes with a network random walk and an 1-alpha percent home teleport home
    w = A.sum(axis = 1).reshape(-1,1)
    n = A.shape[0]
    A = (A + (w == 0)) / (w + n * (w == 0))
    AI = np.linalg.inv(np.eye(n) - A * alpha)
    X = (1 - alpha) * AI
    return X

In [6]:
#auxiliary routines (heritage - unused in this version)
def modularity_matrix(adj): #get modularity matrix from adjacency matrix (heritage)
    w_in = adj.sum(dim=0, keepdim=True)
    w_out = adj.sum(dim=1, keepdim=True)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

def modularity(Q, partition): #get a modularity score for a partition (heritage)
    return (Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum()

def node2vec_embedding(G: nx.Graph, dim=10, walk_length=100, context=10, p=2.0, q=0.5, workers=2, seed=42): #node2vec emebdding to explore as an alternative
    if nx.is_weighted(G):
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1]), edge[2]['weight']) for edge in G.edges(data=True)],
                directed=False, weighted=True)
    else:
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1])) for edge in G.edges(data=True)],
                    directed=False, weighted=False)
    n2v = fastnode2vec.Node2Vec(n2v_graph, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    n2v.train(epochs=100)
    n2v_embeddings = np.array([n2v.wv[str(node)] for node in G])
    return n2v_embeddings

def rpr_clustering(A: np.array, n_clusters=4, kmeans_runs=100, alpha=0.85, seed=42): #cluster network based on our embedding (heritage)
    rpr_embedding = residential_page_rank_embedding(A, alpha)
    X = StandardScaler().fit_transform(X=rpr_embedding)
    rpr_cluster_labels = KMeans(n_clusters=n_clusters, n_init=kmeans_runs, random_state=seed).fit(X).labels_
    return rpr_cluster_labels

def n2v_clustering(G: nx.Graph, n_clusters=4, kmeans_runs=100, dim=10, walk_length=100, context=10, p=2.0, q=0.5, workers=2, seed=42): #cluster network based on node2vec embedding  (heritage)
    n2v_embeddings = node2vec_embedding(G, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    X = StandardScaler().fit_transform(X=n2v_embeddings)
    n2v_cluster_labels = KMeans(n_clusters=n_clusters, n_init=kmeans_runs, random_state=seed).fit(X).labels_
    return n2v_cluster_labels

In [7]:
def aggregateEmbedding(X, n_groups): #produce aggregate embedding through node clustering - visiting one of the n_groups node clusters based on the embedding X, with a network random walk and an 1-alpha percent home teleport home
  alpha = 0.85
  kmeans_runs = 100
  seed = 1
  XS = StandardScaler().fit_transform(X = X)
  model = KMeans(n_clusters = n_groups, n_init=kmeans_runs, random_state=seed).fit(XS)
  #model = GaussianMixture(n_components = n_groups, n_init=kmeans_runs, random_state=seed).fit(X)
  #rpr_cluster_labels = model.predict_proba(X)
  cluster_labels = model.predict(XS)
  XC = model.cluster_centers_
  dist2 = np.array([[((XS[i, :] - XC[j, :])**2).sum() for j in range(n_groups)] for i in range(X.shape[0])])
  sigma2 = dist2.min(axis = 1).mean()
  probs = np.exp(- dist2 / sigma2)
  probs /= probs.sum(axis = 1).reshape(-1,1)
  XA = np.matmul(X, probs)
  return XA

In [8]:
def modularity_matrix_np(adj): #numpy version of modularity matrix
    if not(isinstance(adj, np.ndarray)):
        adj = nx.to_numpy_array(G)
    w_in = adj.sum(axis=0).reshape(-1,1)
    w_out = adj.sum(axis=1).reshape(1,-1)
    T = w_out.sum()
    Q = adj / T - w_out * w_in / T ** 2
    return Q

In [9]:
def clusterEmbeddings(X, n_comms = 2, cluststyle = 'k-means', seed = 1, runs = 100): #cluser nodes by provided embeddings
    if cluststyle == 'GM':
        XS = StandardScaler().fit_transform(X = X)
        model = GaussianMixture(n_components = n_comms, n_init=runs, random_state=seed).fit(XS)
        partition = model.predict(XS)
    else:
        XS = StandardScaler().fit_transform(X = X)
        model = KMeans(n_clusters=n_comms, n_init=runs, random_state=seed).fit(XS)
        partition = model.labels_
    return partition, model

In [10]:
def produceAggEmbed(A, n_agg, alpha = 0.85): #produce and stack together hierarchical embeddings of a given resolutions (list n_agg, 0 stands for original network resulution)
    X = residential_page_rank_embedding(A, alpha = alpha)
    X_ = np.zeros((X.shape[0],0))
    for na in n_agg:
        if na == 0:
            XA = X.copy()
        else:
            XA = aggregateEmbedding(X, n_groups = na)
        X_ = np.concatenate((X_, XA), axis = 1)
    return X_

In [11]:
#Gs =  [(nx.from_numpy_array(nx.to_numpy_array(nx.karate_club_graph(), weight=None)), 'karate', 4)]#, (nx.les_miserables_graph(),'lesmis',6)]

In [12]:
#processNet('karate_34') #karate_34 & 0.419790 & 0.419790 & 0.419790 & 0.419790 & 0.05 & 0.12 & 0.14 & 1.25
#processNet('dolphins_62') #ok dolphins_62 & 0.527728 & 0.526463 & 0.527728 & 0.528519 & 0.11 & 0.15 & 0.10 & 1.13
#processNet('football_115') #ok football_115 & 0.605445 & 0.605445 & 0.605445 & 0.605445 & 0.22 & 0.26 & 0.34 & 2.89
#        processNet('polbooks_105') #ok polbooks_105 & 0.526967 & 0.527237 & 0.527237 & 0.527237 & 0.22 & 0.23 & 0.18 & 1.57
#        processNet('copperfield_112') #ok ccopperfield_112 & 0.303028 & 0.309556 & 0.310173 & 0.313301 & 0.29 & 0.33 & 0.22 & 1.99
#        processNet('email_1133')  # ok - undirected, unweighted email_1133 & 0.573062 & 0.582755 & 0.572484 & 0.578221 & 5.15 & 47.57 & 7.36 & 71.62
#        processNet('lesmis_77') #ok lesmis_77 & 0.566688 & 0.566688 & 0.566688 & 0.566688 & 0.12 & 0.16 & 0.24 & 1.94
#        processNet('celegansneural_297')  # directed celegansneural_297 & 0.503509 & 0.507642 & 0.505710 & 0.507642 & 16.43 & 1.19 & 1.46 & 8.61 | seed = 1: & 0.503509 & 0.507605 & 0.506235 & 0.507642 & 16.32 & 1.09 & 1.45 & 8.26
#        processNet('jazz_198')  # celegansmetabolic_198 & 0.445627 & 0.444787 & 0.445522 & 0.445627 & 6.28 & 0.35 & 0.83 & 5.50
#        processNet('USAir97_332')  # seed = 1; USAir97_332 & 0.207925 & 0.215244 & 0.217149 & 0.217751 & 20.27 & 1.08 & 1.72 & 9.29


In [79]:
#load the selected network

nname = 'lesmis_77' 
#nname = 'copperfield_112'
#nname = 'polbooks_105'
#nname = 'jazz_198' #0.443
#G = loadNetworkMat(nname)
#G = nx.karate_club_graph()
G = nx.les_miserables_graph() #load les miserables
n_comms = 6

print("Network ", nname, '; communities:', n_comms)
A = nx.to_numpy_array(G)
Q = modularity_matrix_np(A)

Network  lesmis_77 ; communities: 6


In [80]:
#produce and partition hierarchical embeddings
#cluststyle = 'GM' 
cluststyle = 'k-means'
seed = 12
X_ = produceAggEmbed(A, n_agg = [15, 6], alpha = 0.85) #stacked hierachical embeddings of given resolutions
N = X_.shape[1]
partition, _ = clusterEmbeddings(X = X_, n_comms = n_comms, cluststyle = cluststyle, seed = seed)
Q = modularity_matrix_np(A)
(Q * (partition.reshape(-1,1) == partition.reshape(1,-1))).sum() #return modularity of the resulting clustering

0.5647873289708507

In [82]:
combo_comms, combo_mod = pycombo.execute(G, max_communities = n_comms); combo_mod #COMBO modularity for comparison

0.566687983343249

In [16]:
#Vanilla neural network
class VNNLayer(nn.Module):
    def __init__(self, in_features, out_features, dropout=0.0):
        super(VNNLayer, self).__init__()
        self.weight1 = nn.Parameter(torch.randn(in_features, out_features)) # 0.5 * torch.eye(in_features, out_features))
        self.bias = nn.Parameter(torch.randn(1, out_features)) # -0.5 * torch.ones(1, out_features))
        self.dropout = dropout
        #xavier initialization
        torch.nn.init.xavier_uniform(self.weight1)
        torch.nn.init.xavier_uniform(self.bias)
        

    def forward(self, input):
        v1 = torch.mm(input, self.weight1)
        output = v1 + self.bias
        output = F.dropout(output, p=self.dropout, training=self.training)
        return output

class VNN_MLP(nn.Module):
    def __init__(self, in_features, out_features, hid_layers = 0, hidden_dim=8, dropout=0.0, SEED = 1):
        #super(VNN_MLP, self).__init__()
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        super().__init__()
        self.n_layers = hid_layers + 1
        self.hidden_dim = hidden_dim
        if self.n_layers > 1:
            layers = [VNNLayer(in_features, self.hidden_dim, dropout)]
        else:
            layers = [VNNLayer(in_features, out_features, dropout)]
        for _ in range(self.n_layers-2):
            layers.append(VNNLayer(self.hidden_dim, self.hidden_dim, dropout))
        #if self.n_layers > 1:
        layers.append(VNNLayer(self.hidden_dim, out_features, dropout = 0))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for i in range(self.n_layers - 1):
            x = self.layers[i](x)
            x = nn.ReLU()(x)
        x = self.layers[-1](x)
        x = nn.Softmax(dim=1)(x)
        #x = 1.0 + x - x.max(dim=-1, keepdim=True).values
        #x = torch.clamp(x, 0, 1)
        #x = x / x.sum(dim=-1, keepdim=True) #normalize st sum = 1
        return x

In [17]:
#network partition class based on VNN
class VNNpartitioner (VNN_MLP):
    def __init__(self, A, X, out_features = 2, hid_layers = 1, hidden_dim = 12, dropout = 0.0, SEED = 1):
        self.adj = torch.FloatTensor(A)
        self.Q = modularity_matrix(self.adj)
        self.features = torch.FloatTensor(X)
        super().__init__(self.features.shape[1], out_features = out_features, hid_layers = hid_layers, hidden_dim = hidden_dim, dropout = dropout, SEED = SEED)
         
    def fitSupervised(self, target_comms, n_epochs = 1000, lr = 0.005, SEED = 1, batchsize = 0): #supervised node classification based on given target labels
        track_best = True
        C0 = torch.tensor(to_categorical(target_comms))
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        t_total = time.time()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(n_epochs):
            batchind = range(self.features.shape[0])
            if batchsize > 0:
                batchind = np.random.choice(batchind, batchsize) #batching
            t_1run = time.time()
            optimizer.zero_grad()
            out_embed = self.forward(self.features[batchind, :])
            C = out_embed[:, :n_comms]
            loss = torch.mean(torch.square(torch.subtract(C,C0[batchind,:]))) #torch.nn.functional.binary_cross_entropy(C, C0)
            loss.backward()
            optimizer.step()
            
            if track_best: #store best states so far
                if batchsize > 0:
                    full_embed = self.forward(self.features)
                    C = full_embed[:, :n_comms]
                    full_loss = torch.mean(torch.square(torch.subtract(C,C0)))
                else:
                    full_loss = loss
            
                if epoch == 0 or full_loss < best_loss:
                    best_loss = full_loss #- torch.trace(Q)
                    best_C = C.data
                    best_embed = out_embed.data
                    best_epoch = epoch
                
            else:
                full_loss = loss
                best_loss = loss
                
            if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
                #optimizer = optim.Adam(model.parameters(), lr=lr)
                print('Epoch: {:04d}'.format(epoch + 1), 'batch MSE: {:.8f}'.format(loss.item()), 'full MSE: {:.8f}'.format(full_loss.item()),
                        'best MSE: {:.8f}'.format(best_loss.item()),
                        'time: {:.4f}s'.format(time.time() - t_1run))
        ent = best_loss.item()
        print("Optimization Finished with MSE ", ent)
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
        return C, ent
    
    def fitUnsupervised(self, n_epochs = 1000, lr = 0.005, SEED = 1, batchsize = 0, restore_best = 500): #unsupervised node embedding clustering optimizing resulting modularity score
        track_best = True
        np.random.seed(SEED)
        torch.manual_seed(SEED)
        t_total = time.time()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        
        out_embed = self.forward(self.features)
        C = out_embed[:, :n_comms].data
        
        for epoch in range(n_epochs):
            t_1run = time.time()
            optimizer.zero_grad()
            batchind = range(self.features.shape[0])
            if batchsize > 0:
                batchind = np.random.choice(batchind, batchsize)
            out_embed = self.forward(self.features[batchind, :])
            C[batchind, :] = out_embed[:, :n_comms]
            Q1 = torch.mm(C.T, self.Q)
            Q2 = torch.mm(Q1, C)
            loss = torch.trace(Q2)
            loss = -loss
            loss.backward()
            optimizer.step()
            C = C.data
            
            if track_best: #store best options so far
                if batchsize > 0:
                    full_embed = self.forward(self.features)
                    C = full_embed[:, :n_comms].data
                    full_loss = - torch.trace(torch.mm(torch.mm(C.T, self.Q), C))
                else:
                    full_loss = loss
            
                if epoch == 0 or full_loss < best_loss:
                    best_loss = full_loss #- torch.trace(Q)
                    best_C = C.data
                    best_embed = out_embed.data
                    best_epoch = epoch
                    
                    torch.save(self.state_dict(), 'model_scripted.pt')
                    
                if (epoch + 1) % restore_best == 0:
                    self.load_state_dict(torch.load('model_scripted.pt'))
                    self.eval()
                
            else:
                full_loss = loss
                best_loss = loss
            
            if n_epochs <= 20 or epoch % (n_epochs//20) == 0 or epoch == n_epochs - 1:
                #optimizer = optim.Adam(model.parameters(), lr=lr)
                print('Epoch: {:04d}'.format(epoch + 1), 'modularity: batch: {:.8f}'.format(-loss.item()), 'full: {:.8f}'.format(-full_loss.item()),
                        'best: {:.8f}'.format(-best_loss.item()),
                        'time: {:.4f}s'.format(time.time() - t_1run))
                #print('Epoch: {:04d}'.format(epoch + 1),
                #        'Modularity: {:.8f}'.format(-best_loss.item()),
                #        'time: {:.4f}s'.format(time.time() - t_1run))
        mod = -best_loss.item()
        print("Optimization Finished with modularity ", mod)
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
        return C, mod
        
        

In [84]:
partitioner = VNNpartitioner(A, X_, out_features = n_comms, hid_layers = 4, hidden_dim = 2 * N, dropout = 0.0, SEED = 1) #init VNN partitioner - 4 hidden layers or input dim neurons each 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  # Remove the CWD from sys.path while we load stuff.


In [85]:
partitioner.fitUnsupervised(n_epochs = 10000, lr = 0.0005, SEED = 1, batchsize = 20, restore_best = 200); #unsupervised node embedding clustering optimizing modularity - stuck early in local maxima if run from scratch

Epoch: 0001 modularity: batch: 0.00003167 full: 0.00003410 best: 0.00003410 time: 0.0079s
Epoch: 0501 modularity: batch: 0.38129368 full: 0.38129500 best: 0.38129500 time: 0.0035s
Epoch: 1001 modularity: batch: 0.38142294 full: 0.38142291 best: 0.38142294 time: 0.0025s
Epoch: 1501 modularity: batch: 0.38143337 full: 0.38143337 best: 0.38143337 time: 0.0025s
Epoch: 2001 modularity: batch: 0.38143638 full: 0.38143641 best: 0.38143641 time: 0.0036s
Epoch: 2501 modularity: batch: 0.38143778 full: 0.38143781 best: 0.38143781 time: 0.0024s
Epoch: 3001 modularity: batch: 0.38143849 full: 0.38143846 best: 0.38143849 time: 0.0024s
Epoch: 3501 modularity: batch: 0.38143894 full: 0.38143894 best: 0.38143897 time: 0.0024s
Epoch: 4001 modularity: batch: 0.38143933 full: 0.38143930 best: 0.38143933 time: 0.0025s
Epoch: 4501 modularity: batch: 0.38143942 full: 0.38143942 best: 0.38143951 time: 0.0026s
Epoch: 5001 modularity: batch: 0.38143948 full: 0.38143948 best: 0.38143951 time: 0.0056s
Epoch: 550

In [86]:
partitioner = VNNpartitioner(A, X_, out_features = n_comms, hid_layers = 4, hidden_dim = 2 * N, dropout = 0.2, SEED = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  # Remove the CWD from sys.path while we load stuff.


In [87]:
partitioner.fitSupervised(partition, n_epochs = 3000, lr = 0.0005, SEED = 1, batchsize = 20); #pre-tune the model using a supervised fit to the previously acheived clustering

Epoch: 0001 batch MSE: 0.14060409 full MSE: 0.13899609 best MSE: 0.13899609 time: 0.0098s
Epoch: 0151 batch MSE: 0.13418409 full MSE: 0.12930012 best MSE: 0.12588826 time: 0.0026s
Epoch: 0301 batch MSE: 0.10195141 full MSE: 0.09144533 best MSE: 0.08998152 time: 0.0028s
Epoch: 0451 batch MSE: 0.05528569 full MSE: 0.05914411 best MSE: 0.05264277 time: 0.0025s
Epoch: 0601 batch MSE: 0.02733142 full MSE: 0.04781006 best MSE: 0.04395728 time: 0.0026s
Epoch: 0751 batch MSE: 0.02459649 full MSE: 0.03292270 best MSE: 0.03089871 time: 0.0025s
Epoch: 0901 batch MSE: 0.00765562 full MSE: 0.01826604 best MSE: 0.01383921 time: 0.0036s
Epoch: 1051 batch MSE: 0.01781948 full MSE: 0.01766050 best MSE: 0.00899935 time: 0.0026s
Epoch: 1201 batch MSE: 0.01918280 full MSE: 0.01404206 best MSE: 0.00608805 time: 0.0026s
Epoch: 1351 batch MSE: 0.00666399 full MSE: 0.00711895 best MSE: 0.00238308 time: 0.0026s
Epoch: 1501 batch MSE: 0.02195257 full MSE: 0.00751295 best MSE: 0.00210181 time: 0.0042s
Epoch: 165

In [88]:
partitioner.dropout = 0.0; #remove dropout for unsupervised fit
partitioner.fitUnsupervised(n_epochs = 20000, lr = 0.0005, SEED = 2, batchsize = 20, restore_best = 500); #unsupervised node embedding clustering optimizing modularity 

Epoch: 0001 modularity: batch: 0.55041718 full: 0.54953468 best: 0.54953468 time: 0.0076s
Epoch: 1001 modularity: batch: 0.56613362 full: 0.56613380 best: 0.56613380 time: 0.0034s
Epoch: 2001 modularity: batch: 0.56623113 full: 0.56623125 best: 0.56623125 time: 0.0045s
Epoch: 3001 modularity: batch: 0.56626582 full: 0.56626582 best: 0.56626582 time: 0.0025s
Epoch: 4001 modularity: batch: 0.56628126 full: 0.56628120 best: 0.56628126 time: 0.0025s
Epoch: 5001 modularity: batch: 0.56628889 full: 0.56628889 best: 0.56628889 time: 0.0027s
Epoch: 6001 modularity: batch: 0.56629312 full: 0.56629312 best: 0.56629312 time: 0.0026s
Epoch: 7001 modularity: batch: 0.56629533 full: 0.56629527 best: 0.56629533 time: 0.0026s
Epoch: 8001 modularity: batch: 0.56629658 full: 0.56629652 best: 0.56629658 time: 0.0025s
Epoch: 9001 modularity: batch: 0.56629729 full: 0.56629723 best: 0.56629729 time: 0.0025s
Epoch: 10001 modularity: batch: 0.56629765 full: 0.56629765 best: 0.56629771 time: 0.0027s
Epoch: 11